In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np 
import os
import cv2
#import skimage.io as io
#import skimage.transform as trans
import numpy as np
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras
# Input data files are available in the read-only "../input/" directory
import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

In [3]:
!kaggle datasets download -d carlolepelaars/camvid

 98% 565M/575M [00:02<00:00, 293MB/s]
100% 575M/575M [00:02<00:00, 268MB/s]


In [ ]:
! unzip camvid.zip

In [5]:
colors = pd.read_csv('/content/CamVid/class_dict.csv').set_index("name")
classes = {cl:list(colors.loc[cl, :]) for cl in colors.index}
classes

{'Animal': [64, 128, 64],
 'Archway': [192, 0, 128],
 'Bicyclist': [0, 128, 192],
 'Bridge': [0, 128, 64],
 'Building': [128, 0, 0],
 'Car': [64, 0, 128],
 'CartLuggagePram': [64, 0, 192],
 'Child': [192, 128, 64],
 'Column_Pole': [192, 192, 128],
 'Fence': [64, 64, 128],
 'LaneMkgsDriv': [128, 0, 192],
 'LaneMkgsNonDriv': [192, 0, 64],
 'Misc_Text': [128, 128, 64],
 'MotorcycleScooter': [192, 0, 192],
 'OtherMoving': [128, 64, 64],
 'ParkingBlock': [64, 192, 128],
 'Pedestrian': [64, 64, 0],
 'Road': [128, 64, 128],
 'RoadShoulder': [128, 128, 192],
 'Sidewalk': [0, 0, 192],
 'SignSymbol': [192, 128, 128],
 'Sky': [128, 128, 128],
 'SUVPickupTruck': [64, 128, 192],
 'TrafficCone': [0, 0, 64],
 'TrafficLight': [0, 64, 64],
 'Train': [192, 64, 128],
 'Tree': [128, 128, 0],
 'Truck_Bus': [192, 128, 192],
 'Tunnel': [64, 0, 64],
 'VegetationMisc': [192, 192, 0],
 'Void': [0, 0, 0],
 'Wall': [64, 192, 0]}

In [6]:
for i in list(classes.values()): 
    print(i)

[64, 128, 64]
[192, 0, 128]
[0, 128, 192]
[0, 128, 64]
[128, 0, 0]
[64, 0, 128]
[64, 0, 192]
[192, 128, 64]
[192, 192, 128]
[64, 64, 128]
[128, 0, 192]
[192, 0, 64]
[128, 128, 64]
[192, 0, 192]
[128, 64, 64]
[64, 192, 128]
[64, 64, 0]
[128, 64, 128]
[128, 128, 192]
[0, 0, 192]
[192, 128, 128]
[128, 128, 128]
[64, 128, 192]
[0, 0, 64]
[0, 64, 64]
[192, 64, 128]
[128, 128, 0]
[192, 128, 192]
[64, 0, 64]
[192, 192, 0]
[0, 0, 0]
[64, 192, 0]


In [7]:
def load_masks(main_path):
    mask = tf.io.read_file(main_path)
    mask = tf.image.decode_png(mask, channels=3)

    # Resize image and mask
    mask = tf.image.resize(mask, size=(256, 256))

    # Convert image and mask to float32 and normalize pixel values
    mask = mask_preprocess(mask)
    return mask

In [8]:
def load_images(main_path):
    image = tf.io.read_file(main_path)
    image = tf.image.decode_jpeg(image, channels=3)

    # Resize image and mask
    image = tf.image.resize(image, size=(256, 256))

    # Convert image and mask to float32 and normalize pixel values
    image = tf.cast(image , tf.float32) / 255
    return image

In [9]:
def mask_preprocess(mask):
    chann = []
    for i in list(classes.values()):
        equali = np.equal(mask , i)
        maski = np.all(equali , axis = -1)
        chann.append(maski)
    chann = np.stack(chann , axis = -1)
    return np.float32(chann)

In [10]:
import os
import numpy as np
from tensorflow.keras.utils import Sequence

class CustomDataGenerator(Sequence):
    def __init__(self, images_dir, masks_dir, batch_size):
        self.images_dir = images_dir
        self.masks_dir = masks_dir
        self.batch_size = batch_size
        self.image_filenames = os.listdir(images_dir)
        self.mask_filenames = os.listdir(masks_dir)
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.image_filenames) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_image_filenames = self.image_filenames[idx*self.batch_size:(idx+1)*self.batch_size]
        batch_mask_filenames = self.mask_filenames[idx*self.batch_size:(idx+1)*self.batch_size]
        batch_images = []
        batch_masks = []
        for image_filename, mask_filename in zip(batch_image_filenames, batch_mask_filenames):
            image_path = os.path.join(self.images_dir, image_filename)
            mask_path = os.path.join(self.masks_dir, mask_filename)
            image = load_images(image_path) # replace with your image loading function
            mask_32_channels = load_masks(mask_path) # apply your function
            batch_images.append(image)
            batch_masks.append(mask_32_channels)
        return np.array(batch_images), np.array(batch_masks)

    def on_epoch_end(self):
        np.random.shuffle(self.image_filenames)
        np.random.shuffle(self.mask_filenames)


In [11]:
generator = CustomDataGenerator(images_dir='/content/CamVid/train', masks_dir='/content/CamVid/train_labels', batch_size=32)

In [12]:
def conv_block(tensor, nfilters, size=3, padding='same', initializer="he_normal"):
    x = Conv2D(filters=nfilters, kernel_size=(size, size), padding=padding, kernel_initializer=initializer)(tensor)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Conv2D(filters=nfilters, kernel_size=(size, size), padding=padding, kernel_initializer=initializer)(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    return x


def deconv_block(tensor, residual, nfilters, size=3, padding='same', strides=(2, 2)):
    y = Conv2DTranspose(nfilters, kernel_size=(size, size), strides=strides, padding=padding)(tensor)
    y = concatenate([y, residual], axis=3)
    y = conv_block(y, nfilters)
    return y
def Unet(h, w, filters, num_classes = 32):
# down
    input_layer = Input(shape=(h, w, 3), name='image_input')
    conv1 = conv_block(input_layer, nfilters=filters)
    conv1_out = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = conv_block(conv1_out, nfilters=filters*2)
    conv2_out = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = conv_block(conv2_out, nfilters=filters*4)
    conv3_out = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = conv_block(conv3_out, nfilters=filters*8)
    conv4_out = MaxPooling2D(pool_size=(2, 2))(conv4)
    conv4_out = Dropout(0.5)(conv4_out)
    conv5 = conv_block(conv4_out, nfilters=filters*16)
    conv5 = Dropout(0.5)(conv5)
# up
    deconv6 = deconv_block(conv5, residual=conv4, nfilters=filters*8)
    deconv6 = Dropout(0.5)(deconv6)
    deconv7 = deconv_block(deconv6, residual=conv3, nfilters=filters*4)
    deconv7 = Dropout(0.5)(deconv7) 
    deconv8 = deconv_block(deconv7, residual=conv2, nfilters=filters*2)
    deconv9 = deconv_block(deconv8, residual=conv1, nfilters=filters)
    output_layer = Conv2D(filters=32, kernel_size=(1, 1), activation='softmax')(deconv9)

    model = Model(inputs=input_layer, outputs=output_layer, name='Unet')
    return model

In [13]:
model = Unet(256 , 256 , 3 , 32)
# model.summary()

In [14]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics='accuracy')

In [15]:
model.fit(generator, steps_per_epoch=len(generator), epochs=10)

Epoch 1/10
12/12 [==============================] - 78s 4s/step - loss: 3.2748 - accuracy: 0.1779
Epoch 2/10
12/12 [==============================] - 48s 4s/step - loss: 3.2030 - accuracy: 0.2060
Epoch 3/10
12/12 [==============================] - 48s 4s/step - loss: 3.1328 - accuracy: 0.2131
Epoch 4/10
12/12 [==============================] - 48s 4s/step - loss: 3.0749 - accuracy: 0.2102
Epoch 5/10
12/12 [==============================] - 48s 4s/step - loss: 3.0178 - accuracy: 0.2101
Epoch 6/10
12/12 [==============================] - 49s 4s/step - loss: 2.9644 - accuracy: 0.2131
Epoch 7/10
12/12 [==============================] - 47s 4s/step - loss: 2.9198 - accuracy: 0.2058
Epoch 8/10
12/12 [==============================] - 48s 4s/step - loss: 2.8639 - accuracy: 0.2086
Epoch 9/10
12/12 [==============================] - 48s 4s/step - loss: 2.8081 - accuracy: 0.2099
Epoch 10/10
12/12 [==============================] - 47s 4s/step - loss: 2.7442 - accuracy: 0.2049
